<a href="https://colab.research.google.com/github/Tifou-Kerdjou/deep-learning/blob/main/analyse_des_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from keras.models import Sequential
from keras.models import Model
import matplotlib.pyplot as plt
from keras.layers import Dense , Dropout , Flatten, BatchNormalization
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense , Dropout , Flatten, Activation, Conv2D, ReLU,GlobalAveragePooling2D,MaxPooling2D,Input
from tensorflow.keras.utils import to_categorical


In [ ]:
tweet=pd.read_csv('sentiment-tweets.csv',error_bad_lines=False)

In [ ]:
tweet.head(100)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,2,0.5000,5,NaN,4049,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,2446,31
1,570301130888122368,positive,0.3486,2,0.0000,5,NaN,5395,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,2446,62
2,570301083672813571,neutral,0.6837,2,0.5000,5,NaN,7678,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,1219,27
3,570301031407624196,negative,1.0000,0,0.7033,5,NaN,5395,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,2446,62
4,570300817074462722,negative,1.0000,1,1.0000,5,NaN,5395,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,2446,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,569910981868060673,negative,1.0000,4,0.6863,5,NaN,2160,NaN,0,"@VirginAmerica Is it me, or is your website do...",NaN,2015-02-23 09:25:41 -0800,1286,10
96,569909224521641984,negative,1.0000,4,0.6771,5,NaN,694,NaN,0,@VirginAmerica I can't check in or add a bag. ...,NaN,2015-02-23 09:18:42 -0800,215,57
97,569907336485019648,negative,1.0000,1,0.6590,5,NaN,2295,NaN,0,@VirginAmerica - Let 2 scanned in passengers l...,NaN,2015-02-23 09:11:12 -0800,2446,62
98,569896805611089920,negative,1.0000,7,0.6714,5,NaN,6040,NaN,0,@virginamerica What is your phone number. I ca...,NaN,2015-02-23 08:29:21 -0800,2256,31


In [ ]:
tweet['tweet_location'].fillna("Unknown",inplace=True)


In [ ]:
print(tweet.isnull().sum())

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                      0
negativereason_confidence           0
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                      0
user_timezone                       0
dtype: int64


In [ ]:
def encode(Y):
    labels =np.unique(Y)
    code= list(range(len(labels)))
    code_label = dict(zip(code,labels))
    label_code = dict(zip(labels,code))
    
    Y_encoded=[]
    for y in Y:
        Y_encoded.append(label_code[y])
    return(np.array(Y_encoded), code_label)

In [ ]:
target,_=encode(tweet['airline_sentiment'])


In [ ]:
tweet['negativereason'],_=encode(tweet['negativereason'])
tweet['airline'],_=encode(tweet['airline'])
tweet['name'],_=encode(tweet['name'])
tweet['user_timezone'],_=encode(tweet['user_timezone'])
tweet['tweet_location'],_=encode(tweet['tweet_location'])

tweet['tweet_created'],_=encode(tweet['tweet_created'])

In [ ]:
data=tweet.drop(columns=['tweet_id','airline_sentiment','text','airline_sentiment_gold','negativereason_gold','tweet_coord']).to_numpy()

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(data, target, test_size=0.3, train_size=0.7, random_state=True)

In [ ]:
x_train[1]


array([6.933e-01, 1.000e+00, 3.560e-01, 3.000e+00, 5.011e+03, 0.000e+00,
       7.824e+03, 2.446e+03, 6.200e+01])

In [ ]:
y_train=np_utils.to_categorical(y_train,3)
y_test=np_utils.to_categorical(y_test,3)

In [ ]:
x_train=x_train.reshape(-1,9,1,1)
x_test=x_test.reshape(-1,9,1,1)

# NN

In [ ]:
def CNN(input_size):
  input = Input(input_size)
  
  x = Conv2D(filters=64, kernel_size = (3,3), activation="relu")(input)
  x = Conv2D(filters=64, kernel_size = (3,3), activation="relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)
  x = BatchNormalization()(x)
  x = Conv2D(filters=128, kernel_size = (3,3), activation="relu")(x)
  x = Conv2D(filters=128, kernel_size = (3,3), activation="relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)
  x = BatchNormalization()(x)  
  x = Conv2D(filters=256, kernel_size = (3,3), activation="relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)
  x = Flatten()(x)
  x = BatchNormalization()(x)
  x = Dense(512,activation="relu")(x)
  output = Dense(3, activation='softmax')(x)
  
  
  model = Model(input, output)
  return model


In [ ]:
batch_size=100
nb_epoch=10
input_size=9,1,1


model=CNN(input_size)
model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=batch_size,epochs=nb_epoch,verbose=1)

ValueError: ignored

In [ ]:
scores=model.evaluate(x_test,y_test)
from sklearn.metrics import confusion_matrix
y_predict=np.argmax(model.predict(x_test),axis=1)
y_test_labels=y_test.argmax(1)
#confusion matrix and classification report 
print("confusion Matrix\n",confusion_matrix(y_test_labels,y_predict))